In [1]:
import torch
import gc

gc.collect()
torch.cuda.empty_cache()

# Query Transformer with Gemma

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

tokenizer = AutoTokenizer.from_pretrained("rtzr/ko-gemma-2-9b-it")
model = AutoModelForCausalLM.from_pretrained(
    "rtzr/ko-gemma-2-9b-it",
    device_map="auto",
    load_in_8bit=True
)


/opt/conda/envs/myvenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 10/10 [00:13<00:00,  1.36s/it]


In [91]:
def generate_search_query(queries):
    message = """
    ## Instructions
    - 당신은 사용자의 여러 대화 메시지를 하나의 검색 쿼리로 변환하는 전문가입니다.
    - 아래 사용자의 대화에서 중요한 정보만 추출하여 간결하고 명확한 검색용 쿼리로 변환하세요.
    - 대화 형식이 아닌, 연구나 조사를 위한 검색어처럼 정확한 질문을 생성하세요.

    ## 대화
    """

    dialogue = [query['content'] for query in queries if query['role'] == 'user']

    prompt = [{"role": "user", "content": message + '\n'.join(dialogue)}]

    chat_prompt = tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(chat_prompt, return_tensors="pt", padding=True).to(device)

    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=200).to(device)
    
    search_query = tokenizer.decode(output[0], skip_special_tokens=True)

    print("-" * 100)
    print(chat_prompt)

    result = [sent for sent in search_query.split('\n') if sent != '']
    return result[-1].strip()




In [92]:
dummy_data = [
    {"eval_id": 44, "msg": [{"role": "user", "content": "같은 독감에 다시 걸리면 빨리 회복이 되더라구?"}, {"role": "assistant", "content": "네 맞습니다."}, {"role": "user", "content": "그 이유가 뭐야?"}]},
    {"eval_id": 278, "msg": [{"role": "user", "content": "노인들이 종종 후각 능력을 잃게 되는 경우가 있더라구?"}, {"role": "assistant", "content": "네 맞습니다."}, {"role": "user", "content": "이렇게 냄새를 못맡게 되면 어떤 결과로 이어지지?"}]},
    {"eval_id": 53, "msg": [{"role": "user", "content": "하늘이 파란 이유는?"}]},
    {"eval_id": 26, "msg": [{"role": "user", "content": "짚신 벌레의 번식은 어떻게 이루어지나?"}]},
]

In [93]:
for test_queries in dummy_data:
    search_query = generate_search_query(test_queries['msg'])
    print(search_query)
    print("-" * 100)

----------------------------------------------------------------------------------------------------
<bos><start_of_turn>user

    ## Instructions
    - 당신은 사용자의 여러 대화 메시지를 하나의 검색 쿼리로 변환하는 전문가입니다.
    - 아래 사용자의 대화에서 중요한 정보만 추출하여 간결하고 명확한 검색용 쿼리로 변환하세요.
    - 대화 형식이 아닌, 연구나 조사를 위한 검색어처럼 정확한 질문을 생성하세요.

    ## 대화
    같은 독감에 다시 걸리면 빨리 회복이 되더라구?
그 이유가 뭐야?<end_of_turn>
<start_of_turn>model

독감 재감염 시 회복 기간 단축 관련 연구 논문
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
<bos><start_of_turn>user

    ## Instructions
    - 당신은 사용자의 여러 대화 메시지를 하나의 검색 쿼리로 변환하는 전문가입니다.
    - 아래 사용자의 대화에서 중요한 정보만 추출하여 간결하고 명확한 검색용 쿼리로 변환하세요.
    - 대화 형식이 아닌, 연구나 조사를 위한 검색어처럼 정확한 질문을 생성하세요.

    ## 대화
    노인들이 종종 후각 능력을 잃게 되는 경우가 있더라구?
이렇게 냄새를 못맡게 되면 어떤 결과로 이어지지?<end_of_turn>
<start_of_turn>model

노인의 후각 감퇴, 원인 및 영향 분석
--------------------------------------------------